### Libraries:

In [2]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib
import json

### Loading CSV file:

In [3]:
#os.chdir("C:/Users/ori/Desktop/txt_for_python/data")

data_file="destinations_LP_crawler_Ex5.csv"
data=pd.read_csv(data_file,encoding=("ISO-8859-1"))
print(data)

         city                           city_LP
0     Antalya       mediterranean-coast/antalya
1      Athens                            Athens
2   Ayia-napa  the-republic-of-cyprus/Agia-napa
3     Bangkok                           Bangkok
4   Barcelona                         Barcelona
..        ...                               ...
71     Venice                            Venice
72     Vienna                            Vienna
73     Warsaw                            Warsaw
74  Zakynthos                         Zakynthos
75     Zurich                            Zurich

[76 rows x 2 columns]


### Geolocation

In [8]:
country={}
#Api key:
api_key ="AIzaSblablablablablalbalbalblablablalblabxoQmNakbU"

for city in data.iloc:
    address=city[0] #City
    url_geo = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
    response = requests.get(url_geo)
    try:
        if not response.status_code == 200:
            print("Error - API request didnt work")
        else:
            response_data=response.json()
            country[address]=response_data['results'][0]['formatted_address'].split(",")[-1]
    except:
        print("problema")
print(country) 
print(len(country))





{'Antalya': ' Turkey', 'Athens': ' Greece', 'Ayia-napa': ' Cyprus', 'Bangkok': ' Thailand', 'Barcelona': ' Spain', 'Basel': ' Switzerland', 'Batumi': ' Georgia', 'Beijing': ' China', 'Berlin': ' Germany', 'Boston': ' USA', 'Bratislava': ' Slovakia', 'Brussels': ' Belgium', 'Bucharest': ' Romania', 'Budapest': ' Hungary', 'Burgas': ' Bulgaria', 'Cancun': ' Mexico', 'Catania': ' Italy', 'Chiang-mai': ' Thailand', 'Cologne': ' Germany', 'Corfu': ' Greece', 'Costa-Brava': ' Spain', 'Crete': ' Greece', 'krakow': ' Poland', 'Delhi': ' India', 'Dresden': ' Germany', 'Dublin': ' Ireland', 'Dubrovnik': ' Croatia', 'Dusseldorf': ' Germany', 'Florence': ' Italy', 'Frankfurt': ' Germany', 'Geneva': ' Switzerland', 'Guangzhou': ' China', 'Hamburg': ' Germany', 'Innsbruck': ' Austria', 'Istanbul': ' Turkey', 'Kos': ' Greece', 'Ko-samui': ' Thailand', 'Kyiv ukrain': ' 02000', 'LarnaKa': ' Cyprus', 'Las-Vegas': ' USA', 'Limassol': ' Cyprus', 'Lisbon': ' Portugal', 'Ljubljana': ' Slovenia', 'Los Angele

### DF with city and country

In [9]:
df=pd.DataFrame.from_dict(country, orient='index')
df.reset_index(inplace=True)
df.columns=["city","country"]
print(df)
df.loc[37,"country"]="ukraine"
df.loc[56,"country"]="ukraine"

         city       country
0     Antalya        Turkey
1      Athens        Greece
2   Ayia-napa        Cyprus
3     Bangkok      Thailand
4   Barcelona         Spain
..        ...           ...
71     Venice         Italy
72     Vienna       Austria
73     Warsaw        Poland
74  Zakynthos        Greece
75     Zurich   Switzerland

[76 rows x 2 columns]


### List of URL and adding discription

In [10]:
discription_list=[]
URL_list=[]
lonely_planet="https://www.lonelyplanet.com/"
for i in range(len(df)):
    url=lonely_planet+df["country"][i].strip()+"/"+data["city_LP"][i]
    URL_list.append(url)
    response_2=requests.get(url)
    if not response_2.status_code == 200:
        print("Error - scrapping didnt work "+url)
        print(i)
    else:
        soup = BeautifulSoup(response_2.content, 'html.parser')
        discription=soup.find("div",class_="readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg").get_text()
        discription_list.append(discription)
        
print(len(discription_list))
df["Description"]=discription_list
print(df)

76
         city       country                                        Description
0     Antalya        Turkey  Once seen simply as the gateway to the Turkish...
1      Athens        Greece  With equal measures of grunge and grace, Athen...
2   Ayia-napa        Cyprus  Endless blue-sky days and a coastline riddled ...
3     Bangkok      Thailand  Same same, but different. This Thailish T-shir...
4   Barcelona         Spain  Barcelona is an enchanting seaside city with b...
..        ...           ...                                                ...
71     Venice         Italy  Imagine the audacity of building a city of mar...
72     Vienna       Austria  Baroque streetscapes and imperial palaces set ...
73     Warsaw        Poland  A phoenix arisen from the ashes, Poland's capi...
74  Zakynthos        Greece  Zakynthos, also known by its Italian name Zant...
75     Zurich   Switzerland  Culturally vibrant, efficiently run and attrac...

[76 rows x 3 columns]
